# **The Battle of Neighbourhoods IBM Capstone Project**

## **Introduction** 

As part of the course, we looked at location data for Toronto and New York, these cities had a lot of replies from the Foursquare API calls, so I decided to look at smaller cities that are expanding and test if a similar comparison can be done with them. The aim is to contribute to future interested parties for example data analysts, Tourists, location data managers, if there are limitations that might need to be resolved using a different API or collecting more data to feed into the requests etc. In this project the comparison is between Toronto and London Ontario as I wanted to have a city which I had to start compiling the data as there was not much location data that was already compiled, but also a city near Toronto so that can also inform anyone wanting to explore further than Toronto example Tourists on other places of interest.

**Toronto** is a multicultural city in the Province of Ontario, it is the most populated city in Canada, it had a population of over 2.7 million in 2016 (Canadian census), also has the highest proportion of minority ethnic population in Canada. Its economy is diversified: Art, Design, Banking & Finance, Life Science, Technology, Aerospace, Fashion, Education, Tourism etc, thus an attractive location for many headquarters of multinational corporations, it welcomes many immigrants annually thus attracting an inflow of talent and opportunities for innovation and expansion as well as having influence on the neighbourhood demographics and tastes. As urban centres grow some people move out to the suburbs to seek other job opportunities or choose to retire elsewhere, others move to be closer to friends and family, others seek a change etc. Where to relocate to depends on many factors for example, job opportunities in their industry, average wages, housing, public/community provisions, crime rates, access to amenities, warmer weather etc.

**London** is a city in Western Ontario which had a population of 383,822 in 2016 according to the Canadian census, it is about 120 miles from Toronto and Detroit. Strong industries are Healthcare with several hospitals located in the city, Education for example the University of Western Ontario (UWO), Insurance and Technology industries. London holds cultural events; Tourists can also explore the settlement changes that happened after the natural disasters and immigration influence over time since the early settlers, for example links to the UK with many similar names and systems. I want to compare Toronto and London Ontario using location data from Foursquare API to look at the different neighbourhoods in these cities, to find the most popular venues in each.

### **Data**

Location data: postcodes, neighbourhood, borough data for Toronto has been taken from a Wikipedia page “List of postal codes of Canada: M - Wikipedia” that was mentioned as part of the course. The data for London Ontario has been taken from another Wikipedia page “List of postal codes of Canada: N - Wikipedia”. 

The Foursquare API will be used to gather data for the different venues to analyse these urban neighbourhoods in Toronto and London Ontario. Longitude and Latitude data for Toronto was provided in week 3 of the course “http://cocl.us/Geospatial_data” and London Ontario geographical coordinates data were gathered from “https://www.geonames.org/”  also geolocator has been used.

Data scrapping and cleaning using python packages. I will cluster the neighbourhoods and choose top 10 most popular venues in each and plot them on a map using Folium to help visualise and interpret the data.

In [10]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
address = 'London Ontario, CA'
geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London Ontario are 42.9836747, -81.2496068.


In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np

import json

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize

import matplotlib.cm as cm

import matplotlib.colors as colors

from sklearn.cluster import KMeans

!pip install folium

print("Libraries Imported")

Libraries Imported


In [2]:
import folium

## Gathering the Data

In [3]:
#Using the information provided in week 3 of the course, creating a dataframe for Toronto which will then be merged with the geo data and the final data will be used to send the API calls. 
Toronto = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df=Toronto[0]
df.columns
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#The data needs to be cleaned to remove missing data, in this case removing "Not assigned" Borough
df=df.drop(index=df[df["Borough"]=="Not assigned"].index)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
#Postcodes with Longitude and Latitude csv file that was provided, making the Postal code as the index can then later merge the two dataframes on Postal code
postcodes=pd.read_csv("http://cocl.us/Geospatial_data", index_col="Postal Code")
postcodes.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [6]:
#set index on the other dataframe as Postal Code then can merge  
df.set_index("Postal Code")

,Borough,Neighbourhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"
M3B,North York,Don Mills
M4B,East York,"Parkview Hill, Woodbine Gardens"


In [7]:
#merge the dataframes
df_Toronto=pd.merge(df,postcodes,on=["Postal Code"])
df_Toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [8]:
#Then plotted the Toronto data using folium, to get a map of Toronto (Fig 1. Map of Toronto)
map_toronto = folium.Map(location=[43.6534817, -79.3839347], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto["Borough"], df_Toronto['Neighbourhood']):
    label = '{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [9]:
CLIENT_ID = '1P2JFELWESBS3BYQJCQVAUCF0UQY0L3HWW210OB24YVWVWWG' 
CLIENT_SECRET = 'XUUWYEL1BG2ID04JRSEABHJY1VH25ZI3E4VL35Q0YUO1AM5R' 
VERSION = '20200605' 
LIMIT = 500 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1P2JFELWESBS3BYQJCQVAUCF0UQY0L3HWW210OB24YVWVWWG
CLIENT_SECRET:XUUWYEL1BG2ID04JRSEABHJY1VH25ZI3E4VL35Q0YUO1AM5R


Venues near the Toronto neighbourhood geodata are called using “explore” Foursquare API request

In [12]:
#gathering Venues data: Foursquare "explore" call
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
                   
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
           
        results = requests.get(url).json()["response"]['groups'][0]['items']
         
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
#Toronto Venues from API call
Toronto_venues = getNearbyVenues(names=df_Toronto["Neighbourhood"],
                                   latitudes=df_Toronto["Latitude"],
                                   longitudes=df_Toronto["Longitude"]
                                   )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [14]:
print(Toronto_venues.shape)
Toronto_venues.head()

(10258, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,Parkwoods,43.753259,-79.329656,Galleria Supermarket,43.753520,-79.349518,Supermarket
3,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café
4,Parkwoods,43.753259,-79.329656,Island Foods,43.745866,-79.346035,Caribbean Restaurant


**The API call returned 10,258 venues in Toronto, a neighbourhood count is done as a snapshot to see if the data is spread across the neighbourhoods.**


In [15]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,100,100,100,100,100,100
"Alderwood, Long Branch",100,100,100,100,100,100
"Bathurst Manor, Wilson Heights, Downsview North",100,100,100,100,100,100
Bayview Village,95,95,95,95,95,95
"Bedford Park, Lawrence Manor East",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Birch Cliff, Cliffside West",100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100


In [16]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 255 uniques categories.


In [17]:
#The venues are assigned against each neighbourhood so that can later be quantified to work out the popular venues
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [18]:
#As noted 10,258 venues were returned across the Toronto neighbourhoods
Toronto_onehot.shape

(10258, 256)

**The mean value per venue is calculated to rank the popularity and to summarise the data, then a snapshot of the top 5 venues per neighbourhood is viewed.**

In [19]:
#The mean value per venue is calculated to rank the popularity and to summarise the data, then a snapshot of the top 5 venues per neighbourhood is viewed
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Campground,Cantonese Restaurant,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Agincourt,0.000,0.00,0.00,0.00,0.010000,0.00,0.000,0.01,0.010000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.030000,0.010000,0.0000,0.000000,0.00,0.0000,0.00,0.000000,0.00,0.000000,0.000,0.000,0.000000,0.020000,0.00,0.0000,0.00,0.00,0.010000,0.0000,0.0000,0.00,0.010000,0.00,0.020000,0.

In [20]:
#top 5 venues
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0           Coffee Shop  0.05
1    Chinese Restaurant  0.05
2  Caribbean Restaurant  0.04
3          Noodle House  0.04
4      Sushi Restaurant  0.03


----Alderwood, Long Branch----
           venue  freq
0         Bakery  0.06
1    Coffee Shop  0.06
2    Pizza Place  0.03
3  Burrito Place  0.03
4   Burger Joint  0.03


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.06
1  Middle Eastern Restaurant  0.04
2          Korean Restaurant  0.03
3              Grocery Store  0.03
4            Bubble Tea Shop  0.03


----Bayview Village----
                       venue  freq
0                Coffee Shop  0.08
1                Supermarket  0.07
2                     Bakery  0.05
3  Middle Eastern Restaurant  0.05
4          Korean Restaurant  0.04


----Bedford Park, Lawrence Manor East----
              venue  freq
0       Coffee Shop  0.09
1              Café  0.07
2    

                venue  freq
0                Café  0.10
1                Park  0.10
2         Coffee Shop  0.09
3  Italian Restaurant  0.08
4       Grocery Store  0.05


----Rouge Hill, Port Union, Highland Creek----
            venue  freq
0     Zoo Exhibit  0.18
1            Park  0.08
2     Coffee Shop  0.06
3     Gas Station  0.04
4  Breakfast Spot  0.03


----Runnymede, Swansea----
         venue  freq
0         Café  0.10
1       Bakery  0.08
2         Park  0.07
3  Coffee Shop  0.07
4      Brewery  0.06


----Runnymede, The Junction North----
                venue  freq
0                Café  0.10
1  Italian Restaurant  0.07
2              Bakery  0.06
3             Brewery  0.06
4         Coffee Shop  0.05


----Scarborough Village----
              venue  freq
0       Coffee Shop  0.08
1      Burger Joint  0.05
2              Park  0.05
3    Ice Cream Shop  0.04
4  Department Store  0.03


----South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Th

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**The top 10 venues are then returned per neighbourhood, the geo data is added back to the data to be able to plot the neighbourhoods.**

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Coffee Shop,Chinese Restaurant,Caribbean Restaurant,Noodle House,Park,Supermarket,Sushi Restaurant,Sandwich Place,Bakery,Pizza Place
1,"Alderwood, Long Branch",Bakery,Coffee Shop,Pizza Place,Park,Liquor Store,Seafood Restaurant,Grocery Store,Burger Joint,Burrito Place,Brewery
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Middle Eastern Restaurant,Clothing Store,Korean Restaurant,Bubble Tea Shop,Grocery Store,Restaurant,Sushi Restaurant,Bakery,Japanese Restaurant
3,Bayview Village,Coffee Shop,Supermarket,Middle Eastern Restaurant,Bakery,Korean Restaurant,Japanese Restaurant,Thai Restaurant,Restaurant,Bagel Shop,Chinese Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Café,Park,Grocery Store,Sushi Restaurant,Bakery,Japanese Restaurant,Liquor Store,Shopping Mall,Thai Restaurant


In [23]:
kclusters = 6

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([3, 3, 3, 3, 1, 2, 1, 4, 1, 2], dtype=int32)

**The data was grouped into 6 clusters as it gave a better distribution, then the clusters are plotted with folium on the Toronto map.**

In [24]:
#venue data and geodata is merged
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = df_Toronto

Toronto_merged = Toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Middle Eastern Restaurant,Grocery Store,Caribbean Restaurant,Burger Joint,Supermarket,Japanese Restaurant,Italian Restaurant,Bakery,Liquor Store,Café
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Park,Supermarket,Burger Joint,Coffee Shop,Italian Restaurant,Grocery Store,Middle Eastern Restaurant,Liquor Store,Pizza Place,Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Park,Café,Pizza Place,Plaza,French Restaurant,Hotel,Neighborhood,Farmers Market,Mediterranean Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Coffee Shop,Italian Restaurant,Vietnamese Restaurant,Café,Bakery,Park,Bank,Mediterranean Restaurant,Brazilian Restaurant,Burger Joint
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Park,Café,Farmers Market,Bakery,Plaza,Sandwich Place,Neighborhood,Restaurant,Liquor Store


**Toronto Cluster Map below show most clusters fit per geographical area**

In [25]:
#map of Toronto clusters
map_cluster = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster)
map_cluster

In [27]:
#Red Cluster
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Malvern, Rouge",Zoo Exhibit,Coffee Shop,Pharmacy,Gas Station,Sandwich Place,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Restaurant,Burger Joint
12,"Rouge Hill, Port Union, Highland Creek",Zoo Exhibit,Park,Coffee Shop,Gas Station,Bank,Pharmacy,Breakfast Spot,Pizza Place,Grocery Store,Pub
95,Upper Rouge,Zoo Exhibit,Sandwich Place,Bank,Fast Food Restaurant,Coffee Shop,Pharmacy,Grocery Store,Pizza Place,Gas Station,Liquor Store


In [31]:
#Purple Cluster
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,"Lawrence Manor, Lawrence Heights",Coffee Shop,Italian Restaurant,Vietnamese Restaurant,Café,Bakery,Park,Bank,Mediterranean Restaurant,Brazilian Restaurant,Burger Joint
8,"Parkview Hill, Woodbine Gardens",Park,Beach,Café,Bakery,BBQ Joint,Brewery,Grocery Store,Middle Eastern Restaurant,Coffee Shop,Thai Restaurant
14,Woodbine Heights,Coffee Shop,Brewery,Café,Park,Gastropub,BBQ Joint,Bakery,Farmers Market,Indian Restaurant,Beach
19,The Beaches,Park,Coffee Shop,Beach,Bakery,Brewery,Ice Cream Shop,Café,Breakfast Spot,Pub,Gastropub
35,"East Toronto, Broadview North (Old East York)",Park,Coffee Shop,Brewery,Café,Bakery,BBQ Joint,Beach,Pizza Place,Gastropub,Farmers Market
41,"The Danforth West, Riverdale",Park,Coffee Shop,Café,Brewery,Greek Restaurant,Bakery,Farmers Market,Gastropub,Historic Site,Ice Cream Shop
44,"Golden Mile, Clairlea, Oakridge",Park,Beach,Coffee Shop,Grocery Store,Breakfast Spot,Sandwich Place,Burger Joint,Café,Pub,Middle Eastern Restaurant
47,"India Bazaar, The Beaches West",Park,Coffee Shop,Brewery,Café,Beach,Bakery,BBQ Joint,Gastropub,Pizza Place,Ice Cream Shop
54,Studio District,Coffee Shop,Park,Brewery,Café,Farmers Market,Beach,Bakery,Ice Cream Shop,Greek Restaurant,Hotel
55,"Bedford Park, Lawrence Manor East",Coffee Shop,Café,Park,Grocery Store,Sushi Restaurant,Bakery,Japanese Restaurant,Liquor Store,Shopping Mall,Thai Restaurant


In [32]:
#Blue Cluster, all are in Downtown Toronto
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Regent Park, Harbourfront",Coffee Shop,Park,Café,Pizza Place,Plaza,French Restaurant,Hotel,Neighborhood,Farmers Market,Mediterranean Restaurant
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Park,Café,Farmers Market,Bakery,Plaza,Sandwich Place,Neighborhood,Restaurant,Liquor Store
9,"Garden District, Ryerson",Coffee Shop,Park,Café,Farmers Market,Plaza,Hotel,Restaurant,Bakery,Liquor Store,French Restaurant
15,St. James Town,Coffee Shop,Park,Café,Farmers Market,Plaza,Bakery,Hotel,Historic Site,Sandwich Place,Liquor Store
20,Berczy Park,Coffee Shop,Park,Café,Bakery,Farmers Market,Plaza,Hotel,Neighborhood,French Restaurant,Historic Site
24,Central Bay Street,Coffee Shop,Park,Café,Bakery,Farmers Market,Plaza,Sandwich Place,Historic Site,Dessert Shop,Mexican Restaurant
30,"Richmond, Adelaide, King",Coffee Shop,Park,Café,Bakery,Farmers Market,Sandwich Place,Plaza,Restaurant,Gastropub,Historic Site
36,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Park,Café,Bakery,Plaza,Farmers Market,Historic Site,French Restaurant,Hotel,Mexican Restaurant
42,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Park,Café,Farmers Market,Bakery,Plaza,Restaurant,Hotel,Historic Site,French Restaurant
48,"Commerce Court, Victoria Hotel",Coffee Shop,Park,Café,Farmers Market,Bakery,Plaza,Mexican Restaurant,French Restaurant,Historic Site,Restaurant


In [33]:
#Turquoise Cluster
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Middle Eastern Restaurant,Grocery Store,Caribbean Restaurant,Burger Joint,Supermarket,Japanese Restaurant,Italian Restaurant,Bakery,Liquor Store,Café
1,Victoria Village,Park,Supermarket,Burger Joint,Coffee Shop,Italian Restaurant,Grocery Store,Middle Eastern Restaurant,Liquor Store,Pizza Place,Restaurant
7,Don Mills,Park,Middle Eastern Restaurant,Café,Italian Restaurant,Supermarket,Bakery,Coffee Shop,Grocery Store,Liquor Store,Japanese Restaurant
11,"West Deane Park, Princess Gardens, Martin Grov...",Coffee Shop,Bakery,Sandwich Place,Grocery Store,Liquor Store,Seafood Restaurant,Restaurant,Hotel,Golf Course,Japanese Restaurant
13,Don Mills,Park,Middle Eastern Restaurant,Café,Italian Restaurant,Supermarket,Bakery,Coffee Shop,Grocery Store,Liquor Store,Japanese Restaurant
17,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Grocery Store,Coffee Shop,Hotel,Bakery,Liquor Store,Sandwich Place,Sporting Goods Shop,Eastern European Restaurant,Department Store,Breakfast Spot
27,Hillcrest Village,Chinese Restaurant,Bakery,Japanese Restaurant,Caribbean Restaurant,Coffee Shop,Supermarket,Korean Restaurant,Greek Restaurant,Middle Eastern Restaurant,Thai Restaurant
28,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Middle Eastern Restaurant,Clothing Store,Korean Restaurant,Bubble Tea Shop,Grocery Store,Restaurant,Sushi Restaurant,Bakery,Japanese Restaurant
32,Scarborough Village,Coffee Shop,Burger Joint,Park,Ice Cream Shop,Pharmacy,Supermarket,Indian Restaurant,Department Store,Gym / Fitness Center,Discount Store
33,"Fairview, Henry Farm, Oriole",Chinese Restaurant,Bakery,Caribbean Restaurant,Liquor Store,Supermarket,Coffee Shop,Pharmacy,Restaurant,Japanese Restaurant,Ice Cream Shop


In [34]:
#Green Cluster
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Glencairn,Café,Coffee Shop,Park,Brewery,Italian Restaurant,Bakery,Liquor Store,Grocery Store,Indian Restaurant,Tea Room
16,Humewood-Cedarvale,Café,Coffee Shop,Park,Italian Restaurant,Bar,Grocery Store,Bakery,Dessert Shop,Gastropub,Middle Eastern Restaurant
21,Caledonia-Fairbanks,Café,Coffee Shop,Brewery,Park,Italian Restaurant,Bar,Bakery,Middle Eastern Restaurant,Indian Restaurant,Caribbean Restaurant
23,Leaside,Park,Café,Italian Restaurant,Coffee Shop,Bakery,Grocery Store,Bookstore,BBQ Joint,Brewery,Other Great Outdoors
25,Christie,Café,Coffee Shop,Park,Bakery,Italian Restaurant,Grocery Store,Sandwich Place,Spa,Bar,Theater
29,Thorncliffe Park,Park,Café,Italian Restaurant,Coffee Shop,Bakery,Grocery Store,BBQ Joint,Pizza Place,Spa,Brewery
31,"Dufferin, Dovercourt Village",Café,Coffee Shop,Bar,Park,Italian Restaurant,Brewery,Ice Cream Shop,Indian Restaurant,Restaurant,Bakery
37,"Little Portugal, Trinity",Park,Café,Coffee Shop,Bakery,Sandwich Place,Farmers Market,Gastropub,Hotel,Restaurant,Grocery Store
43,"Brockton, Parkdale Village, Exhibition Place",Park,Café,Coffee Shop,Bakery,Sandwich Place,Bar,Pizza Place,Plaza,Dog Run,Restaurant
62,Roselawn,Café,Park,Coffee Shop,Italian Restaurant,Grocery Store,Brewery,Indian Restaurant,Liquor Store,BBQ Joint,Supermarket


In [35]:
#Orange Cluster 
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,"Islington Avenue, Humber Valley Village",Coffee Shop,Sandwich Place,Brewery,Grocery Store,Bakery,Pharmacy,Pizza Place,Liquor Store,Italian Restaurant,Restaurant
18,"Guildwood, Morningside, West Hill",Coffee Shop,Bank,Sandwich Place,Park,Pharmacy,Restaurant,Indian Restaurant,Fast Food Restaurant,Fried Chicken Joint,Ice Cream Shop
22,Woburn,Coffee Shop,Pharmacy,Park,Pizza Place,Fast Food Restaurant,Hotel,Sandwich Place,Restaurant,Grocery Store,Breakfast Spot
26,Cedarbrae,Coffee Shop,Sandwich Place,Pizza Place,Fast Food Restaurant,Bank,Restaurant,Pharmacy,Fried Chicken Joint,Caribbean Restaurant,Ice Cream Shop
50,Humber Summit,Coffee Shop,Hotel,Sandwich Place,Pharmacy,Indian Restaurant,Steakhouse,Italian Restaurant,Chinese Restaurant,Burger Joint,Bank
51,"Cliffside, Cliffcrest, Scarborough Village West",Coffee Shop,Park,Grocery Store,Sandwich Place,Pharmacy,Gym,Pizza Place,Japanese Restaurant,Clothing Store,Bank
57,"Humberlea, Emery",Coffee Shop,Pizza Place,Vietnamese Restaurant,Gas Station,Fast Food Restaurant,Bank,Pharmacy,Grocery Store,Chinese Restaurant,Asian Restaurant
70,Westmount,Coffee Shop,Sandwich Place,Bank,Pizza Place,Bakery,Restaurant,Golf Course,Gas Station,Chinese Restaurant,Hotel
77,"Kingsview Village, St. Phillips, Martin Grove ...",Coffee Shop,Hotel,Restaurant,Pharmacy,Sandwich Place,Bakery,Bank,Chinese Restaurant,Golf Course,Italian Restaurant
89,"South Steeles, Silverstone, Humbergate, Jamest...",Coffee Shop,Fast Food Restaurant,Sandwich Place,Indian Restaurant,Italian Restaurant,Caribbean Restaurant,Chinese Restaurant,Pharmacy,Asian Restaurant,Sushi Restaurant


**London Ontario** data was then collected from GeoNames for co-ordinates and neighbourhood data,  List of postal codes of Canada: N - Wikipedia for postcodes and neighbourhood data and checking against City Map Gallery (london.ca). 
London Ontario data was not provided by the course and a search on the internet did not result in a tidy table that could be used to read the data and create the dataframe.

In [36]:
#London Ontario dataframe creating by using data collected from geonames and wikipedia, these weren't provided by the course and a search on the internet didnt result in a tidy table that 
#could be used to read the data and create the dataframe, hence this was made below.
LondonOntario={
            "Postal Code":["N6B","N5W","N6A","N6C", "N6H","N6E","N6P", "N6N", "N6M", "N6L", "N6K", "N6J", "N6G", "N5V", "N5Z", "N5Y", "N5X"],
            "Borough":["London Central","London East", "London North", "London South", "London West", "London", "London","London", "London", "London", "London", "London", "London", "London", "London", "London", "London"], 
            "Neighbourhood":["London Central","London East", "London North", "London South", "London West","South White Oaks,Central Westminster,East Longwoods,West Brockley","Talbot,Lambeth,West Tempo,South Sharon Creek","South Highbury,Glanworth,East Brockley,SE Westminster","Jackson,Old Victoria,Bradley,North Highbury","East Tempo","Riverbend,Woodhull,North Sharon Creek,Byron,West Westmount","Southcrest,East Westmount,West Highland","Sunningdale,West Masonville,Medway,NE Hyde Park,East Fox Hollow","YXU,North and East Argyle,East Huron Heights","Glen Cairn","West Huron Heights,Carling","Fanshawe,Stoneybrook,Stoney Creek,Uplands,East Masonville"],
            "Latitude":["42.984","42.986","42.998","42.958","42.991","42.918","42.891","42.9","42.963","42.872","42.954","42.955","43.015","43.023","42.966","43.012","43.044"],
            "Longitude":["-81.239","-81.182","-81.256","-81.238","-81.34","-81.224","-81.324","-81.159","-81.139","-81.247","-81.342","-81.273","-81.305","-81.164","-81.205","-81.231","-81.239"]
}

df_W=pd.DataFrame(LondonOntario)
df_W

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,N6B,London Central,London Central,42.984,-81.239
1,N5W,London East,London East,42.986,-81.182
2,N6A,London North,London North,42.998,-81.256
3,N6C,London South,London South,42.958,-81.238
4,N6H,London West,London West,42.991,-81.34
5,N6E,London,"South White Oaks,Central Westminster,East Long...",42.918,-81.224
6,N6P,London,"Talbot,Lambeth,West Tempo,South Sharon Creek",42.891,-81.324
7,N6N,London,"South Highbury,Glanworth,East Brockley,SE West...",42.9,-81.159
8,N6M,London,"Jackson,Old Victoria,Bradley,North Highbury",42.963,-81.139
9,N6L,London,East Tempo,42.872,-81.247


**The London map plotted with the data above shows the locations plotted are scattered across the city and this is expected as only one location data was plotted per neighbourhood.**

In [38]:
map_london = folium.Map(location=[42.9836747, -81.2496068], zoom_start=12)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_W['Latitude'], df_W['Longitude'], df_W["Borough"], df_W['Neighbourhood']):
    label = '{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  

map_london

In [39]:
#London venues using the “explore" Foursquare API are then sorted and grouped like was done with Toronto data
London_venues = getNearbyVenues(names=df_W["Neighbourhood"],
                                   latitudes=df_W["Latitude"],
                                   longitudes=df_W["Longitude"]
                                 )

London Central
London East
London North
London South
London West
South White Oaks,Central Westminster,East Longwoods,West Brockley
Talbot,Lambeth,West Tempo,South Sharon Creek
South Highbury,Glanworth,East Brockley,SE Westminster
Jackson,Old Victoria,Bradley,North Highbury
East Tempo
Riverbend,Woodhull,North Sharon Creek,Byron,West Westmount
Southcrest,East Westmount,West Highland
Sunningdale,West Masonville,Medway,NE Hyde Park,East Fox Hollow
YXU,North and East Argyle,East Huron Heights
Glen Cairn
West Huron Heights,Carling
Fanshawe,Stoneybrook,Stoney Creek,Uplands,East Masonville


In [40]:
print(London_venues.shape)
London_venues.head()

(1214, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,London Central,42.984,-81.239,Edgar + Joe's Cafe,42.979983,-81.243563,Café
1,London Central,42.984,-81.239,The Morrissey House,42.985865,-81.241693,Pub
2,London Central,42.984,-81.239,TG's Addis Ababa Restaurant,42.987210,-81.237187,African Restaurant
3,London Central,42.984,-81.239,The Works Gourmet Burger Bistro,42.982151,-81.249710,Restaurant
4,London Central,42.984,-81.239,Abruzzi Ristorante,42.982105,-81.250735,Italian Restaurant


**Only 1,214 London venues were returned, this is 11% of the Toronto API replies and this is not surprising as the population in London was about 14% of that of Toronto in 2016.
Also the geo data used to send the request was not the same size as London is a smaller city, this also could explain why some neighbourhoods have only a few venues returned. Another reason could be more geodata needs to be added or the location data needs to be updated**

In [41]:
London_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
East Tempo,4,4,4,4,4,4
"Fanshawe,Stoneybrook,Stoney Creek,Uplands,East Masonville",81,81,81,81,81,81
Glen Cairn,100,100,100,100,100,100
"Jackson,Old Victoria,Bradley,North Highbury",25,25,25,25,25,25
London Central,100,100,100,100,100,100
London East,100,100,100,100,100,100
London North,100,100,100,100,100,100
London South,100,100,100,100,100,100
London West,90,90,90,90,90,90


In [42]:
print('There are {} uniques categories.'.format(len(London_venues['Venue Category'].unique())))

There are 129 uniques categories.


**Sorting the data:** venues are then displayed by the type and then grouped by the mean so that the most common venue can be grouped together like was done with the Toronto data.

In [43]:
London_onehot = pd.get_dummies(London_venues[['Venue Category']], prefix="", prefix_sep="")

London_onehot['Neighbourhood'] = London_venues['Neighbourhood'] 

fixed_columns = [London_onehot.columns[-1]] + list(London_onehot.columns[:-1])
London_onehot = London_onehot[fixed_columns]

London_onehot.head()

,Neighbourhood,African Restaurant,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Store,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Café,Camera Store,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General College & University,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Laser Tag,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Outdoor Supply Store,Outlet Store,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Record Shop,Rental Service,Restaurant,Road,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Ski Area,Ski Lodge,Soccer Field,Soup Place,Sporting Goods Shop,Sports Bar,State / Provincial Park,Storage Facility,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Bar,Yoga Studio
0,London Central,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,London Central,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,London Central,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,London Central,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,London Central,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [44]:
London_grouped = London_onehot.groupby('Neighbourhood').mean().reset_index()
London_grouped

,Neighbourhood,African Restaurant,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Store,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Café,Camera Store,Campground,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Event Space,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General College & University,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Historic Site,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Laser Tag,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Outdoor Supply Store,Outlet Store,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pool,Portuguese Restaurant,Pub,Record Shop,Rental Service,Restaurant,Road,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Ski Area,Ski Lodge,Soccer Field,Soup Place,Sporting Goods Shop,Sports Bar,State / Provincial Park,Storage Facility,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wine Bar,Yoga Studio
0,East Tempo,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.00,0.00,0.000000,0.00,0.250000,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00
1,"Fanshawe,Stoneybrook,Stoney Creek,Uplands,East...",0.00,0.012346,0.012346,0.012346,0.024691,0.00,0.000000,0.000000,0.024691,0.00,0.012346,0.00,0.012346,0.00,0.012346,0.000000,0.024691,0.000000,0.000000,0.012346,0.00,0.000000,0.012346,0.012346,0.012346,0.000000,0.000000,0.00,0.012346,0.074074,0.000000,0.000000,0.000000,0.012346,0.000000,0.012346,0.000000,0.000000,0.00,0.024691,0.000000,0.00,0.012346,0.012346,0.000000,0.000000,0.000000,0.012346,0.00,0.012346,0.000000,0.000000,0.024691,0.024691,0.012346,0.000000,0.000000,0.000000,0.049383,0.024691,0.012346,0.000000,0.012346,0.00,0.012346,0.024691,0.00,0.00,0.000000,0.00,0.012346,0.00000,0.00,0.00,0.000000,0.012346,0.024691,0.000000,0.012346,0.000000,0.00,0.000000,0.012346,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012346,0.024691,0.037037,0.012346,0.000000,0.012346,0.000000,0.049383,0.000000,0.000000,0.086420,0.000000,0.000000,0.000000,0.012346,0.012346,0.000000,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.000000,0.012346,0.000000,0.012346,0.00,0.012346,0.00,0.000000,0.012346,0.000000,

In [45]:
num_top_venues = 5

for hood in London_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = London_grouped[London_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----East Tempo----
                venue  freq
0               Hotel  0.25
1  Italian Restaurant  0.25
2         Golf Course  0.25
3      Breakfast Spot  0.25
4           Irish Pub  0.00


----Fanshawe,Stoneybrook,Stoney Creek,Uplands,East Masonville----
           venue  freq
0     Restaurant  0.09
1    Coffee Shop  0.07
2            Pub  0.05
3  Grocery Store  0.05
4       Pharmacy  0.04


----Glen Cairn----
           venue  freq
0    Coffee Shop  0.11
1     Restaurant  0.04
2  Grocery Store  0.04
3   Liquor Store  0.04
4           Café  0.03


----Jackson,Old Victoria,Bradley,North Highbury----
         venue  freq
0  Coffee Shop  0.16
1     Pharmacy  0.08
2  Gas Station  0.08
3  Supermarket  0.08
4        Diner  0.04


----London Central----
         venue  freq
0  Coffee Shop  0.06
1          Pub  0.05
2         Café  0.04
3    Gastropub  0.03
4      Brewery  0.03


----London East----
                  venue  freq
0           Coffee Shop  0.14
1  Fast Food Restaurant  0.06
2    

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = London_grouped['Neighbourhood']

for ind in np.arange(London_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(London_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Tempo,Hotel,Italian Restaurant,Golf Course,Breakfast Spot,Diner,Discount Store,Dog Run,Eastern European Restaurant,Dessert Shop,Electronics Store
1,"Fanshawe,Stoneybrook,Stoney Creek,Uplands,East...",Restaurant,Coffee Shop,Pub,Grocery Store,Pharmacy,Breakfast Spot,Electronics Store,Bank,Ice Cream Shop,Liquor Store
2,Glen Cairn,Coffee Shop,Grocery Store,Liquor Store,Restaurant,Gym,Pizza Place,Diner,Italian Restaurant,Brewery,Café
3,"Jackson,Old Victoria,Bradley,North Highbury",Coffee Shop,Supermarket,Gas Station,Pharmacy,Grocery Store,Intersection,Flower Shop,Fast Food Restaurant,Movie Theater,Dog Run
4,London Central,Coffee Shop,Pub,Café,Liquor Store,Park,Brewery,Gastropub,Pizza Place,Vegetarian / Vegan Restaurant,Italian Restaurant


In [60]:
kclusters = 6

London_grouped_clustering = London_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(London_grouped_clustering)

kmeans.labels_[0:10] 

array([2, 1, 4, 3, 4, 1, 4, 4, 1, 1], dtype=int32)

In [56]:
#London Venues data merged with geodata and clustered
London_merged = df_W

London_merged = London_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

London_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,N6B,London Central,London Central,42.984,-81.239,1,Coffee Shop,Pub,Café,Liquor Store,Park,Brewery,Gastropub,Pizza Place,Vegetarian / Vegan Restaurant,Italian Restaurant
1,N5W,London East,London East,42.986,-81.182,1,Coffee Shop,Fast Food Restaurant,Gas Station,Pharmacy,Grocery Store,Gym,Restaurant,Liquor Store,Bank,Sandwich Place
2,N6A,London North,London North,42.998,-81.256,1,Pub,Coffee Shop,Park,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Grocery Store,Restaurant,Indian Restaurant
3,N6C,London South,London South,42.958,-81.238,1,Coffee Shop,Restaurant,Café,Park,Italian Restaurant,Pizza Place,Liquor Store,Grocery Store,Pub,Middle Eastern Restaurant
4,N6H,London West,London West,42.991,-81.34,1,Coffee Shop,Grocery Store,Park,Fast Food Restaurant,Pizza Place,Golf Course,Gym,Pharmacy,Supermarket,Sandwich Place


In [61]:
#London Ontario Cluster Map
map_clusters = folium.Map(location=[42.9836747, -81.2496068 ], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(London_merged['Latitude'], London_merged['Longitude'], London_merged['Neighbourhood'], London_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=6,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters 

**London Ontario Cluster Map** the clusters also fit per geographical area: the purple cluster is centrally located to the north where the Education and Healthcare industry is based, also the Airport. Three other smaller clusters are in the south of the city

In [63]:
#Red Cluster
London_merged.loc[London_merged['Cluster Labels'] == 0, London_merged.columns[[2] + list(range(6, London_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,"South Highbury,Glanworth,East Brockley,SE West...",Coffee Shop,Breakfast Spot,Sandwich Place,Waste Facility,Warehouse Store,Clothing Store,Thrift / Vintage Store,Motorcycle Shop,Fast Food Restaurant,Gas Station


In [64]:
#Purple Cluster, this is most of the city 
London_merged.loc[London_merged['Cluster Labels'] == 1, London_merged.columns[[2] + list(range(6, London_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,London Central,Coffee Shop,Pub,Café,Liquor Store,Park,Brewery,Gastropub,Pizza Place,Vegetarian / Vegan Restaurant,Italian Restaurant
1,London East,Coffee Shop,Fast Food Restaurant,Gas Station,Pharmacy,Grocery Store,Gym,Restaurant,Liquor Store,Bank,Sandwich Place
2,London North,Pub,Coffee Shop,Park,Café,Italian Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Grocery Store,Restaurant,Indian Restaurant
3,London South,Coffee Shop,Restaurant,Café,Park,Italian Restaurant,Pizza Place,Liquor Store,Grocery Store,Pub,Middle Eastern Restaurant
4,London West,Coffee Shop,Grocery Store,Park,Fast Food Restaurant,Pizza Place,Golf Course,Gym,Pharmacy,Supermarket,Sandwich Place
5,"South White Oaks,Central Westminster,East Long...",Coffee Shop,Restaurant,Sporting Goods Shop,American Restaurant,Pizza Place,Grocery Store,Gym,Sandwich Place,Japanese Restaurant,Hotel
10,"Riverbend,Woodhull,North Sharon Creek,Byron,We...",Coffee Shop,Golf Course,Grocery Store,Park,Pharmacy,Gym,Pizza Place,Restaurant,Bank,Italian Restaurant
11,"Southcrest,East Westmount,West Highland",Park,Coffee Shop,Pizza Place,Italian Restaurant,Café,Restaurant,Vegetarian / Vegan Restaurant,Grocery Store,Gym / Fitness Center,Seafood Restaurant
12,"Sunningdale,West Masonville,Medway,NE Hyde Par...",Coffee Shop,Grocery Store,Restaurant,Pub,Convenience Store,Yoga Studio,Café,Ice Cream Shop,Gym,Park
13,"YXU,North and East Argyle,East Huron Heights",Coffee Shop,Restaurant,Diner,Hotel,Skating Rink,Campground,Chinese Restaurant,Portuguese Restaurant,Pharmacy,Pet Store


In [65]:
#Blue Cluster
London_merged.loc[London_merged['Cluster Labels'] == 2, London_merged.columns[[2] + list(range(6, London_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East Tempo,Hotel,Italian Restaurant,Golf Course,Breakfast Spot,Diner,Discount Store,Dog Run,Eastern European Restaurant,Dessert Shop,Electronics Store


In [66]:
#Turquoise Cluster
London_merged.loc[London_merged['Cluster Labels'] == 3, London_merged.columns[[2] + list(range(6, London_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,"Talbot,Lambeth,West Tempo,South Sharon Creek",Pharmacy,Gym,Grocery Store,Pub,Coffee Shop,Farmers Market,Bank,Gas Station,Sandwich Place,Beer Store
8,"Jackson,Old Victoria,Bradley,North Highbury",Coffee Shop,Supermarket,Gas Station,Pharmacy,Grocery Store,Intersection,Flower Shop,Fast Food Restaurant,Movie Theater,Dog Run


### Conclusion